Evan Callia, Kyle Kusuda, Zen, Denny, Allie

In [15]:
#Importing appropriate modules
import sys
sys.path.append('C:\Python27\ArcGIS10.3\Lib\site-packages')
import pg
from pg import DB
import csv

In [16]:
#connecting to databse
db = DB(dbname = 'geog465_evcallia', host = 'geog-db2.geog.uw.edu', port = 5432, user = 'evcallia', passwd = 'evcallia')

Creating tables in databse

Constraints are left out becasue they will have to be removed when populationg the tables. They will be added later

In [30]:
db.query('DROP TABLE Permits')

In [31]:
#Permits
db.query('CREATE TABLE Permits (id int PRIMARY KEY, PermitNumber int, Description varchar(500), Value money, ApplicantName varchar(75), ApplicationDate varchar(25), IssueDate varchar(25), FinalDate varchar(25), ExpirationDate varchar(25), Contractor varchar(200), URL varchar(500), MasterNumber varchar(25), Statuses_id int, Permit_Types_id int, Categories_id int, Work_Types_id int, Location_id int, Actions_id int)')

In [4]:
#Actions
db.query('CREATE TABLE Actions (id int PRIMARY KEY, ActionType varchar(75))')

In [6]:
#Categories
db.query('CREATE TABLE Categories (id int PRIMARY KEY, Category varchar(25))')

In [7]:
#Location
db.query('CREATE TABLE Location (id int PRIMARY KEY, Address varchar(75), Latitude varchar(20), Longitude varchar(20), Location varchar(35))')

In [9]:
#PermitTypes
db.query('CREATE TABLE PermitTypes (id int PRIMARY KEY, PermitType varchar(25))')

In [10]:
#Statuses
db.query('CREATE TABLE Statuses (id int PRIMARY KEY, Status varchar(50))') 

In [11]:
#WorkTypes
db.query('CREATE TABLE WorkTypes (id int PRIMARY KEY, WorkType varchar(25))')

In [38]:
db.query('DROP TABLE LinkTable')

In [39]:
#LinkTable 
db.query('CREATE TABLE LinkTable (id int PRIMARY KEY, PermitNumber int, ActionType varchar(75), Category varchar(25), Address varchar(75), PermitType varchar(25), Status varchar(50), WorkType varchar(25))')

Populating Tables

In [40]:
#make lsit of csvs and tables to interate through
csv_list = ['permits.csv', 'actions.csv', 'categories.csv', 'permit_types.csv', 'location.csv', 'statuses.csv', 'work_types.csv', 'Building_Permits_Current.csv']
table_list = ['Permits', 'Actions', 'Categories', 'PermitTypes', 'Location', 'Statuses', 'WorkTypes', 'LinkTable']
index_counter = 0
id_count = 1
for csv_item in csv_list:
    f = open('C:\Users\zenmoore\Desktop\evans_files-2016-05-24\evans_files\\' + csv_item)
    reader = csv.reader(f, delimiter = ',')
    all_rows_list = []
    next(reader, None) #skip header row
    for row in reader:
        if csv_item != 'Building_Permits_Current.csv':   #contidional statement to see if LinkTable
            row_list = []
            for index in row:
                row_list.append(index)
            all_rows_list.append(row_list)
        else:
            #this populates the LinkTable which is a special case and indexes must be hard coded
            all_rows_list.append([id_count, row[0], row[5], row[4], row[2], row[1], row[13], row[6]])
            id_count += 1
    f.close()
    db.query('DELETE FROM ' + table_list[index_counter])
    db.inserttable(table_list[index_counter], all_rows_list)
    index_counter += 1

Populating foreign keys

In [50]:
#FK field Statuses_id
db.query('UPDATE Permits SET Statuses_id = Statuses.id FROM Statuses, linkTable WHERE linkTable.Status = Statuses.Status AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [51]:
#FK field Permit_Types_id
db.query('UPDATE Permits SET Permit_Types_id = PermitTypes.id FROM PermitTypes, linkTable WHERE linkTable.PermitType = PermitTypes.PermitType AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [52]:
#FK field Categories_id
db.query('UPDATE Permits SET Categories_id = Categories.id FROM Categories, linkTable WHERE linkTable.Category = Categories.Category AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [53]:
#FK field Work_Types_id
db.query('UPDATE Permits SET Work_Types_id = WorkTypes.id FROM WorkTypes, linkTable WHERE linkTable.WorkType = WorkTypes.WorkType AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [54]:
#FK field Location_id
db.query('UPDATE Permits SET Location_id = Location.id FROM Location, linkTable WHERE linkTable.Address = Location.Address AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [55]:
#FK field Actions_id
db.query('UPDATE Permits SET Actions_id = Actions.id FROM Actions, linkTable WHERE linkTable.ActionType = Actions.ActionType AND linkTable.PermitNumber = Permits.PermitNumber')

'50301'

In [56]:
print db.query('select Statuses_id, Permit_Types_id, Categories_id, Work_Types_id, Location_id, Actions_id from Permits where id < 11')

statuses_id|permit_types_id|categories_id|work_types_id|location_id|actions_id
-----------+---------------+-------------+-------------+-----------+----------
          1|              1|            1|            1|          1|         1
          2|              2|            2|            1|          2|         2
          2|              2|            2|            1|          3|         2
          2|              2|            3|            1|          4|         2
          2|              2|            4|            1|          5|         2
          2|              2|            3|            1|          6|         2
          1|              1|            5|            1|          7|         3
          2|              2|            2|            1|          8|         2
          2|              2|            2|            1|          9|         2
          2|              2|            2|            1|         10|         2
(10 rows)


Add constriants back to tables

In [57]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Statuses_id) REFERENCES Statuses(id)')

In [62]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Permit_Types_id) REFERENCES PermitTypes(id)')

In [63]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Categories_id) REFERENCES Categories(id)')

In [65]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Work_Types_id) REFERENCES WorkTypes(id)')

In [66]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Location_id) REFERENCES Location(id)')

In [67]:
db.query('ALTER TABLE Permits ADD FOREIGN KEY (Actions_id) REFERENCES Actions(id)')